# Import module

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3

In [2]:
# 載入自訂義檔案, 地址轉經緯度function、googlemap網址生成function
from Function_list import get_latitude_longitude
from Function_list import place_url

# Cinema infomation webcraw

In [3]:
def dataset_webcraw(url):
    response = requests.get(url)
    res = BeautifulSoup(response.text) # 預設html.parser，不打也相同
    latest_data = res.find_all('div', class_ = 'download-item')[-1].a['href']
    return latest_data
url = 'https://data.gov.tw/dataset/22213'
latest_data = dataset_webcraw(url)

# Data Clean & Feature

In [ ]:
df = pd.read_csv(latest_data)
cinema_data = df[['事業名稱','地址']]
cinema_data.dropna(inplace=True)

In [ ]:
# 臺 轉 台
cinema_data['地址'] = cinema_data['地址'].apply(lambda x: '台' + x[1:] if x.startswith('臺') else x)

# 換順序
cinema_data = cinema_data.reindex(columns = ['事業名稱', '地址'])

## 地址錯誤手動修正


In [6]:
# 時代戲院	台中市清水區中山路339號3F~5F
cinema_data.loc[cinema_data['事業名稱'] == '時代戲院', '地址'] = '台中市清水區中山路339號3F~5F'

In [7]:
# 新增經緯度，按下去就會刷google API 哦!
cinema_data['經緯度'] = cinema_data['地址'].map(get_latitude_longitude)

In [8]:
# 經緯度turple轉string
cinema_data['經緯度'] = cinema_data['經緯度'].map(lambda x:str(x).strip('()'))

In [10]:
# 新增GoogleMAP連結
cinema_data['googlemap網址'] = cinema_data.apply(place_url, axis = 1)

# Save to DataBase

In [13]:
# 不帶index 轉DB
conn = sqlite3.connect('TW_cinema_info.db')
cinema_data.to_sql('TW_cinema_info_latest', con=conn, if_exists='replace', index=False)
conn.close()

# 載入DB

In [14]:
conn = sqlite3.connect('TW_cinema_info.db')
data_df = pd.read_sql_query("SELECT * FROM TW_cinema_info_latest", conn)
conn.close()

# Check

In [15]:
data_df

,事業名稱,地址,經緯度,googlemap網址
0,國賓大戲院,台北市成都路88號,"25.042998, 121.50454",https://www.google.com/maps/search/?api=1&quer...
1,台北新光影城,台北市萬華區西寧南路36號4樓,"25.0452169, 121.506424",https://www.google.com/maps/search/?api=1&quer...
2,樂聲大戲院,台北市萬華區武昌街2段85號1~5樓,"25.0452854, 121.5048364",https://www.google.com/maps/search/?api=1&quer...
3,今日秀泰影城,台北市峨嵋街52號4.6.8樓,"25.0329636, 121.5654268",https://www.google.com/maps/search/?api=1&quer...
4,喜滿客絕色電影城,台北市萬華區漢中街52號8至11樓,"25.0438973, 121.5071127",https://www.google.com/maps/search/?api=1&quer...
...,...,...,...,...
110,花蓮新天堂樂園威秀影城,花蓮縣吉安鄉南濱路一段503號3-4樓,"23.9383814, 121.595437",https://www.google.com/maps/search/?api=1&quer...
111,台東秀泰影城,台東縣台東市新生路93號3樓,"22.7523536, 121.147811",https://www.google.com/maps/search/?api=1&quer...
112,金獅影城,金門縣金湖鎮中山路8-5號西棟3樓,"24.4417138, 118.3630416",https://www.google.com/maps/search/?api=1&quer...
113,國賓影城@金門昇恆昌金湖廣場,金門縣金湖鎮太湖路二段198號6樓,"24.4392162, 118.4186129",https://www.google.com/maps/search/?api=1&quer...
